## "런타임 모두 실행"
### 해외리그 5개, 해외국가대항리그 2개, 국내리그 2개에 대한 경기 일정과 결과를 크롤링하여 각각 엑셀 파일로 저장한다.

### ※ 단순오류 시 "이후 셀 실행"으로 오류 코드부터 재시작

### ※ 경로문제 시 해외, 국내축구 페이지 이동 코드부터 재시작

In [ ]:
# 구글 드라이브 연동.
from google.colab import drive
drive.mount('/content/drive')

# sportslight 폴더로 경로 지정.
%cd /content/drive/MyDrive/sportslight

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/sportslight


### 크롤링 작업 전 환경설정

In [ ]:
## 구글 코랩에서 한글 문제 해결
# 코드 설치 => 런타임 다시 시작 => import 다시 실행

# 나눔 글꼴 패키지를 설치.
!sudo apt-get install -y fonts-nanum
# 글꼴 캐시를 업데이트.
!sudo fc-cache -fv
# Matplotlib의 캐시 디렉토리를 삭제.
!rm ~/.cache/matplotlib -rf

# 최종적으로 Matplotlib에서 나눔 글꼴을 사용할 수 있음.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [ ]:
## 동적 크롤링을 위한 Selenium 라이브러리 설치, Chromium 브라우저와 드라이버 설치

# %%shell : 주피터 노트북에서 셸 명령어를 실행하기 위한 매직 명령어
%%shell

# Debian buster의 저장소를 sources.list 파일에 추가.
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# 각각 키를 받아와서 apt 키링에 추가.
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# 각각 키를 가져와서 GPG 키링 파일로 내보냄.
apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# chromium.pref 파일을 생성하고, Debian 저장소에서 chromium* 패키지를 우선 사용하도록 설정.
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# 패키지 목록을 업데이트.
apt-get update

# Chromium 브라우저와 드라이버를 설치.
apt-get install chromium chromium-driver

# Python 패키지 관리자 pip를 사용하여 Selenium 라이브러리를 설치.
pip install selenium

Executing: /tmp/apt-key-gpghome.hfI2DqaUHN/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.jGwXn0UMPQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.hn0zc5uVE1/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
# sleep()과 같은 시간 관련 함수를 제공하는 time 모듈을 가져옴.
import time
# Selenium 라이브러리에서 webdriver 모듈을 가져옴.
from selenium import webdriver
# 웹 페이지에서 요소를 찾는 데 사용되는 검색 방법을 지정하는 데 사용되는 By 클래스를 가져옴.
from selenium.webdriver.common.by import By
# ChromeDriver 실행 파일을 시작하는 데 사용되는 Service 클래스를 가져옴.
from selenium.webdriver.chrome.service import Service  
# 특정 조건이 충족될 때까지 기다리는 데 사용되는 WebDriverWait 클래스를 가져옴.
from selenium.webdriver.support.ui import WebDriverWait
# 모듈을 EC로 가져와서, 요소의 존재 또는 표시 여부와 같은 여러 조건에서 대기할 수 있음.
from selenium.webdriver.support import expected_conditions as EC
# 웹 페이지에서 요소를 찾을 수 없는 경우 발생하는 NoSuchElementException 클래스를 가져옴.
from selenium.common.exceptions import NoSuchElementException    
# Excel 파일로 작업하는 데 사용되는 서드파티 라이브러리인 openpyxl 모듈을 가져옴.
import openpyxl

In [ ]:
# Colab에서는 웹 브라우저 창이 뜨지 않으므로 별도 설정.

# Chrome 웹 드라이버에서 사용할 옵션을 설정하는 ChromeOptions 객체를 생성.
options = webdriver.ChromeOptions()
# --headless 옵션을 추가하여 Chrome 브라우저를 headless 모드로 실행.(GUI 없이 백그라운드에서 브라우저를 실행하는 모드)
options.add_argument('--headless')    
# --no-sandbox 옵션을 추가하여 Chrome 브라우저를 보안 모드로 실행.
options.add_argument('--no-sandbox')
# --disable-dev-shm-usage 옵션을 추가하여 브라우저가 /dev/shm을 사용하지 않도록 함.(/dev/shm : 브라우저가 사용하는 임시 저장소로 사용)
options.add_argument('--disable-dev-shm-usage')
# 설정된 옵션을 사용하여 Chrome 웹 드라이버를 로드하고, driver 객체를 생성.
driver = webdriver.Chrome('chromedriver', options=options)

### 일정/결과 크롤링 작업 시작

In [ ]:
## 네이버 해외축구 일정/결과 페이지로 이동

# 네이버 url로 이동
url = "https://www.naver.com/"
print("네이버 페이지로 이동\n")
driver.get(url)

# 스포츠 페이지로 이동
sports = driver.find_element(By.CSS_SELECTOR, '#NM_NEWSSTAND_HEADER > div.direct_area > a:nth-child(3)')
print(sports.text + " 페이지로 이동\n")
sports.click()

# 해외축구 페이지로 이동
wfootball = driver.find_element(By.CSS_SELECTOR, '#lnb_list > li:nth-child(5) > a')
print(wfootball.text + " 페이지로 이동\n")
wfootball.click()

# 일정/결과 페이지로 이동
schedule = driver.find_element(By.CSS_SELECTOR, '#lnb_sub_list > li:nth-child(4) > a > span')
print(schedule.text + " 페이지로 이동\n")
schedule.click()

print("이동 완료\n")

네이버 페이지로 이동

스포츠 페이지로 이동

해외축구 페이지로 이동

일정/결과 페이지로 이동

이동 완료



In [ ]:
# 엑셀 파일 생성
wb = openpyxl.Workbook()

# 현재 워크시트 선택
# ws = wb.active

# Sheet 워크시트 삭제
# wb.remove_sheet(wb['Sheet'])

# ws = f"variable_{i}"

# 워크시트 생성
ws = wb.create_sheet("matchSchedule")
ws['A1'] = 'scheduleNum'
ws['B1'] = 'leagueName'
ws['C1'] = 'matchDate'
ws['D1'] = 'leagueYM'
ws['E1'] = 'matchTime'
ws['F1'] = 'matchPlace'
ws['G1'] = 'teamAName'
ws['H1'] = 'teamAPoint'
ws['I1'] = 'teamBPoint'
ws['J1'] = 'teamBName'
ws['K1'] = 'matchContent'

In [ ]:
# 프리미어리그 페이지로 이동
epl = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(1) > a > span')
print(epl.text + " 페이지로 이동\n")
epl.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :

    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text
    # print(month_name)

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기.

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')
    # for element_list in match_list 
    #   print(element_list.text)

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # # 일정 고유 번호
      # schedule = int(i)

      # 리그 이름
      epl = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(1) > a > span')
      league = epl.text
      # league_num = league_dict.get(league, "")

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(epl.text + " 페이지 크롤링 완료\n")

# wb.save('./schedule_PL.xlsx')

프리미어리그 페이지로 이동

2022년 일정으로 이동
8월 경기 수 : 60 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 46 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 69 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 45 경기
11월 일정 크롤링 완료

12월 일정으로 이동
12월 경기 수 : 39 경기
12월 일정 크롤링 완료

2023년 일정으로 이동
1월 경기 수 : 55 경기
1월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 54 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 51 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 71 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 59 경기
5월 일정 크롤링 완료

프리미어리그 페이지 크롤링 완료



In [ ]:
# 라리가 페이지로 이동
primera = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(2) > a > span')
print(primera.text + " 페이지로 이동\n")
primera.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :

    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      primera = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(2) > a > span')
      league = primera.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(primera.text + " 페이지 크롤링 완료\n")

라리가 페이지로 이동

2022년 일정으로 이동
8월 경기 수 : 49 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 49 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 68 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 43 경기
11월 일정 크롤링 완료

12월 일정으로 이동
12월 경기 수 : 37 경기
12월 일정 크롤링 완료

2023년 일정으로 이동
1월 경기 수 : 54 경기
1월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 51 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 50 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 66 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 73 경기
5월 일정 크롤링 완료

6월 일정으로 이동
6월 경기 수 : 39 경기
6월 일정 크롤링 완료

라리가 페이지 크롤링 완료



In [ ]:
# 분데스리가 페이지로 이동
bundesliga = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(3) > a > span')
print(bundesliga.text + " 페이지로 이동\n")
bundesliga.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      bundesliga = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(3) > a > span')
      league = bundesliga.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(bundesliga.text + " 페이지 크롤링 완료\n")

분데스리가 페이지로 이동

2022년 일정으로 이동
8월 경기 수 : 55 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 48 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 61 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 49 경기
11월 일정 크롤링 완료

2023년 일정으로 이동
1월 경기 수 : 50 경기
1월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 52 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 49 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 60 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 57 경기
5월 일정 크롤링 완료

분데스리가 페이지 크롤링 완료



In [ ]:
# 세리에 A 페이지로 이동
seria = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(4) > a > span')
print(seria.text + " 페이지로 이동\n")
seria.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      seria = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(4) > a > span')
      league = seria.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'
      
      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(seria.text + " 페이지 크롤링 완료\n")

세리에 A 페이지로 이동

2022년 일정으로 이동
8월 경기 수 : 54 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 54 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 60 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 52 경기
11월 일정 크롤링 완료

2023년 일정으로 이동
1월 경기 수 : 62 경기
1월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 50 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 51 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 62 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 67 경기
5월 일정 크롤링 완료

6월 일정으로 이동
6월 경기 수 : 39 경기
6월 일정 크롤링 완료

세리에 A 페이지 크롤링 완료



In [ ]:
# 리그 1 페이지로 이동
ligue1 = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(5) > a > span')
print(ligue1.text + " 페이지로 이동\n")
ligue1.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      ligue1 = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(5) > a > span')
      league = ligue1.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(ligue1.text + " 페이지 크롤링 완료\n")

리그 1 페이지로 이동

2022년 일정으로 이동
8월 경기 수 : 58 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 61 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 66 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 44 경기
11월 일정 크롤링 완료

12월 일정으로 이동
12월 경기 수 : 38 경기
12월 일정 크롤링 완료

2023년 일정으로 이동
1월 경기 수 : 62 경기
1월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 66 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 52 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 62 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 63 경기
5월 일정 크롤링 완료

6월 일정으로 이동
6월 경기 수 : 39 경기
6월 일정 크롤링 완료

리그 1 페이지 크롤링 완료



In [ ]:
# 챔피언스리그 페이지로 이동
champs = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(6) > a > span')
print(champs.text + " 페이지로 이동\n")
champs.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      champs = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(6) > a > span')
      league = champs.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(champs.text + " 페이지 크롤링 완료\n")

챔피언스리그 페이지로 이동

2022년 일정으로 이동
9월 경기 수 : 58 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 73 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 44 경기
11월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 32 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 35 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 34 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 31 경기
5월 일정 크롤링 완료

챔피언스리그 페이지 크롤링 완료



In [ ]:
# 유로파리그 페이지로 이동
europa = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(7) > a > span')
print(europa.text + " 페이지로 이동\n")
europa.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      europa = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(7) > a > span')
      league = europa.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      if int(d_month) > 7:  # 7월에는 모든 해외축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
        year = "2022"
      else:
        year = "2023"

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(europa.text + " 페이지 크롤링 완료\n")

유로파리그 페이지로 이동

2022년 일정으로 이동
9월 경기 수 : 59 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 76 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 45 경기
11월 일정 크롤링 완료

2월 일정으로 이동
2월 경기 수 : 42 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 45 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 36 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 33 경기
5월 일정 크롤링 완료

유로파리그 페이지 크롤링 완료



### 네이버 국내축구 페이지 이동 후 크롤링 시작

In [ ]:
# 국내축구 페이지로 이동
kfootball = driver.find_element(By.CSS_SELECTOR, '#lnb_list > li:nth-child(4) > a')
print("국내" + kfootball.text + " 페이지로 이동\n")
kfootball.click()

# 일정 / 결과 페이지로 이동
schedule = driver.find_element(By.CSS_SELECTOR, '#lnb_sub_list > li:nth-child(4) > a > span')
print(schedule.text + " 페이지로 이동\n")
schedule.click()

print("이동 완료\n")

국내축구 페이지로 이동

일정/결과 페이지로 이동

이동 완료



In [ ]:
# K리그1 페이지로 이동
kleague = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(1) > a > span')
print(kleague.text + " 페이지로 이동\n")
kleague.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      kleague = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(1) > a > span')
      league = kleague.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      # 1월에는 모든 국내축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
      if int(d_month) > 1:
        year = "2023"
      else:
        year = ""

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(kleague.text + " 페이지 크롤링 완료\n")

K리그1 페이지로 이동

2023년 일정으로 이동
2월 경기 수 : 32 경기
2월 일정 크롤링 완료

3월 일정으로 이동
3월 경기 수 : 43 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 54 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 50 경기
5월 일정 크롤링 완료

6월 일정으로 이동
6월 경기 수 : 46 경기
6월 일정 크롤링 완료

7월 일정으로 이동
7월 경기 수 : 50 경기
7월 일정 크롤링 완료

8월 일정으로 이동
8월 경기 수 : 44 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 44 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 37 경기
10월 일정 크롤링 완료

K리그1 페이지 크롤링 완료



In [ ]:
# K리그2 페이지로 이동
kleague2 = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(2) > a > span')
print(kleague2.text + " 페이지로 이동\n")
kleague2.click()

time.sleep(1) # 1초 동안 대기

# 한 시즌(12개월 치) 경기 일정 데이터가 존재한다고 가정 (데이터가 존재하지 않을 경우 for문 계속 진행)
for i in range (1, 13) :
  try :
    month_list = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span.month')

    month_name = month_list.text

    # n월 페이지로 이동
    month = driver.find_element(By.CSS_SELECTOR, '#_monthList > li:nth-child('+str(i)+') > a > span')
    print(month.text + " 일정으로 이동")
    month.click()

    time.sleep(1) # 1초 동안 대기

    # 월별 스케줄 리스트 찾기
    match_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr')

    time.sleep(1) # 1초 동안 대기

    # n월 총 경기 수 출력
    print(month_name + " 경기 수 : " + str(len(match_list)) + " 경기")

    # num_match에 총 경기 수 담아주기
    num_match = len(match_list)

    # num_match : 이 달의 총 경기 수
    for i in range(1, num_match + 1):

      # 리그 이름
      kleague2 = driver.find_element(By.CSS_SELECTOR, '#_categoryList > li:nth-child(2) > a > span')
      league = kleague2.text

      # 경기 날짜 "yyyy-mm-dd"
      date_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > th > div')
      for date_element in date_list:
        date = date_element.text

      date_parts = date.split('.')  # 문자열을 점을 기준으로 분리하여 리스트에 저장
      d_month = date_parts[0]  # 분리된 첫 번째 요소가 월
      day_parts = date_parts[1].split(' (')  # 두 번째 요소에서 공백과 괄호를 기준으로 다시 분리하여 리스트에 저장
      d_day = day_parts[0]  # 분리된 첫 번째 요소가 일

      # 1월에는 모든 국내축구 모든 시즌이 종료 되어 경기가 존재 하지 않음
      if int(d_month) > 1:
        year = "2023"
      else:
        year = ""

      formatted_date = year + '-' + d_month.zfill(2) + '-' + d_day.zfill(2)

      # 경기 날짜 "yy년 mm월"
      formatted_y = year[-2:]  # year 변수의 마지막 두 자리로 formatted_y에 연도를 저장
      formatted_ym = formatted_y + '년 ' + d_month.zfill(2) + '월'

      # 경기 시간
      matchTime_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(1)')
      if len(matchTime_list) == 0:
        matchTime = ''
      else:
        for matchTime_element in matchTime_list :
            matchTime = matchTime_element.text

      # 경기 장소
      place_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.time_place > div > span:nth-child(2)')
      if len(place_list) == 0:
        place = '경기가 없습니다.'
      else:
        for place_element in place_list :
            place = place_element.text

      # 홈 이름
      home_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.name')
      if len(home_list) == 0:
        home = ''
      else:
        for home_element in home_list :
            home = home_element.text

      # 홈 득점
      homescore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_left > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_left > span.score')
      if len(homescore_list) == 0:
        homescore = ''
      else:
        for homescore_element in homescore_list :
            homescore = homescore_element.text

      # 어웨이 득점
      awayscore_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.score, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.score')
      if len(awayscore_list) == 0:
        awayscore = ''
      else:
        for awayscore_element in awayscore_list :
            awayscore = awayscore_element.text

      # 어웨이 이름
      away_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(3) > div > span.team_right > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right.winner > span.name, #_monthlyScheduleList > tr:nth-child('+str(i)+') > td:nth-child(2) > div > span.team_right > span.name')
      if len(away_list) == 0:
        away = ''
      else:
        for away_element in away_list :
            away = away_element.text

      # 경기 내용
      result_list = driver.find_elements(By.CSS_SELECTOR, '#_monthlyScheduleList > tr:nth-child('+str(i)+') > td.game_content.last > div > span')
      if len(result_list) == 0:
        result = ''
        if len(matchTime_list) == 0:
          result = 'NM'
      else:
        for result_element in result_list :
            result = result_element.text

      ws.append(["", league, formatted_date, formatted_ym, matchTime, place, home, homescore, awayscore, away, result])

    print(month_name + " 일정 크롤링 완료\n")
  except NoSuchElementException :
    continue

print(kleague2.text + " 페이지 크롤링 완료\n")

wb.save('./matchSchedule.xlsx')

print("matchSchedule.xlsx로 저장 완료\n")

K리그2 페이지로 이동

2023년 일정으로 이동
3월 경기 수 : 48 경기
3월 일정 크롤링 완료

4월 일정으로 이동
4월 경기 수 : 54 경기
4월 일정 크롤링 완료

5월 일정으로 이동
5월 경기 수 : 50 경기
5월 일정 크롤링 완료

6월 일정으로 이동
6월 경기 수 : 42 경기
6월 일정 크롤링 완료

7월 일정으로 이동
7월 경기 수 : 52 경기
7월 일정 크롤링 완료

8월 일정으로 이동
8월 경기 수 : 47 경기
8월 일정 크롤링 완료

9월 일정으로 이동
9월 경기 수 : 48 경기
9월 일정 크롤링 완료

10월 일정으로 이동
10월 경기 수 : 46 경기
10월 일정 크롤링 완료

11월 일정으로 이동
11월 경기 수 : 39 경기
11월 일정 크롤링 완료

K리그2 페이지 크롤링 완료

matchSchedule.xlsx로 저장 완료



In [ ]:
# 종료
driver.quit()